In [ ]:
# setting the Thesis project folder as working directory
%cd "../.."

In [ ]:
import random
import wandb
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import os
from datetime import datetime
from skimage import io
from skimage.exposure import equalize_adapthist
import sys
from utility_functions.utility import *

# Defining functions

Load and log

Things to check each run:
- project name
- artifact name
- artifact description

In [ ]:
def load_and_log(WI_args, config, prj_name="Thesis", entity='lorenzob'):

    # 🚀 start a run, with a type to label it and a project it can call home
    with wandb.init(project=prj_name, job_type="load-data", entity=entity) as run:
           
        datasets = WI_args['img'], WI_args['annot']  # separate code for loading the datasets
        names = ["images", "annotations"]

        # 🏺 create our Artifact
        raw_data = wandb.Artifact(
            "img3_test_set", type="dataset",
            description="new annotations (2.1-3.2)",
            metadata={'z_channel': config['z_channel'],
                      'rgb_channel': config['rgb_channel'],
                      'original_img_shape': WI_args['img'].shape,
                      'max_X': np.max(WI_args['img']),
                      'min_X': np.min(WI_args['img']),
                      'norm':config['norm'],
                      'OTHER':config['other'],
                    #   'focus_dict': config['focus_dict'],
                    },
                    )

        for name, data in zip(names, datasets):
            # 🐣 Store a new file in the artifact, and write something into its contents.
            with raw_data.new_file(name + ".npy", mode="wb") as file:
                np.save(file, data)

        # ✍️ Save the artifact to W&B.
        run.log_artifact(raw_data)

# Setting default parameters and seeds

In [ ]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# plots sizes
plt.rcParams['figure.figsize'] = [10, 10]

In [ ]:
config = {
    'z_channel':(0, 1, 2, 3), 
    'rgb_channel':(0, 1), # 
    # 'focus_dict': 'No', #{'func': None, 'img_size': None, 'step_ij': None},
    'norm': {'M':'skimage adapthist',
        'N':'skimage adapthist', #{'step_ij': None, 'img_size':None, 'treshold': None,}
        },
    'dataset': 'white-mad-hatters/Team-Artifacts-Tesi/GBC-2images-EqAdapthist-raw:v4',
    'other': None,
}

In [ ]:
wandb.login()

# Image loading and cropping

To check:
- path of rgb_img
- path of ann

In [ ]:
rgb_img = io.imread('input/Microglia_Multichannel_20X_-1.tif')[:, :, 1218:, :-651] # 
ann = io.imread('input/Ann3.tif')[ 1218:, :-651]

# assigning the wanted 2D slice of the image to variable
img = rgb_img[config['z_channel'], :, :, :][ :, config['rgb_channel'], :, :, ]

In [ ]:
X, Y = download_data(project_name='Team-Artifacts-Tesi', artifact_path=config['dataset'], entity='white-mad-hatters',  ann_filename='annot2.npy', img_filename='green_blue_fusion2.npy')

In [ ]:
ann = Y
img = X

In [ ]:
ann.shape, img.shape

# Preprocess

Projection of microglia and nuclei

In [ ]:
row_start = 2816
row_end = X.shape[0]
config['other'] ={"row_start": row_start,
                    'row_end': row_end}
row_start, row_end

In [ ]:
# m_prj = np.max(img[:, 1, :, :], axis=0)
m_prj = img[row_start:row_end, :, 0]

m_prj.shape

In [ ]:
# n_prj = np.max(img[:, 0, :, :], axis=0)
n_prj = img[row_start:row_end, :, 1]

n_prj.shape

Annotation norm

In [ ]:
# for Y
print(np.unique(ann))
ann = (ann/np.max(ann)).astype(np.int8)
print(np.unique(ann))
ann = ann[row_start:row_end, :]
print(ann.shape)

In [ ]:
plt.imshow(n_prj)

In [ ]:
m_prj.shape, ann.shape

In [ ]:
coord = 0
step = 1000

plt.imshow(m_prj[coord:coord+step, coord:coord+step])
plt.show()
plt.imshow(ann[coord:coord+step, coord:coord+step])

Normalized image

In [ ]:
m_norm = m_prj

In [ ]:
n_norm = n_prj

In [ ]:
# m_norm = equalize_adapthist(m_prj)

In [ ]:
# n_norm = equalize_adapthist(n_prj)

In [ ]:
np.unique(n_norm), np.unique(m_norm)

# Final dataset

In [ ]:
img_3D = np.full((ann.shape[0], ann.shape[1], 2), -1, dtype=np.float32)
img_3D[:, :, 0] = m_norm
img_3D[:, :, 1] = n_norm

In [ ]:
np.sum(img_3D== -1)

In [ ]:
plt.imshow(m_prj)

In [ ]:
plt.imshow(img_3D[1500:2000, 3000:3500, 0])
plt.show()
plt.imshow(img_3D[1500:2000, 3000:3500, 1])
plt.show()
plt.imshow(ann[1500:2000, 3000:3500])

# Load and log (.npy) to W&B

To check:
- img
- annot

In [ ]:
WI_args = dict(img=img_3D,
               annot=ann)

In [ ]:
load_and_log(WI_args, config, prj_name='Team-Artifacts-Tesi', entity='white-mad-hatters')